In [ ]:
import numba
import numpy as np
a = np.random.rand(16*100_000)
b = np.random.rand(16*100_000)

# Numba `parallel=True` a `prange`
Minule jsme si už ukázali, že numba nabízí automatický paralelismus (pokud to jde).

Našemu kódu můžeme pomoci a říct mu, kde má paralelizovat for cyklus pomocí `prange` (paralel range).

Numba umožňuje paralelní výpočty v "shared memory" modelu (OpenMP). Existuje také možnost využití GPU pomocí CUDA (https://numba.readthedocs.io/en/stable/cuda/index.html). My se ale zaměříme na CPU.

- Koplilace s využitím paralelizace lze docílit parametrem `parallel=True`.
- Veškeré Numpy vektorové operacu budou automaticky zparalelizované.
- `for` cykly lze paralelizovat pomocí `numba.prange` napísto `range`.
    - udělá se automatická distribuce iterací mezi vlákny.
    - pozor na rozdíl mezi `a = a + 1` a `a += 1`, první počítá s tím, že každý thread má své `a`, druhý se chová k `a` jako proměnné do které se redukuje (tedy lze udělat suma skrze všechny vlákna v `prange`).

In [ ]:
@numba.jit(nopython=True, parallel=True)
def my_dot_numba2(a, b):
    result = 0
    for i in numba.prange(a.shape[0]):
        result += a[i] * b[i]
    return result

In [ ]:
%timeit c = my_dot_numba2(a, b)

In [ ]:
%timeit c = np.dot(a, b)

In [ ]:
# možná bude třeba
#!pip install --upgrade numba

In [ ]:
from numba import jit, prange
import numpy as np

@jit(nopython = True, parallel=True)
def test(x):
    n = x.shape[0]
    a = np.sin(x)
    b = np.cos(a * a)
    acc = 0
    for i in prange(n - 2):
        for j in prange(n - 1):
            acc += b[i] + b[j + 1]
    return acc


In [ ]:
test(np.arange(10))


Numba umožňuje inspekci paralelizace kódu pomocí `.parallel_diagnostics()`.

In [ ]:

test.parallel_diagnostics(level=4)

## Dot product z minula

In [ ]:
import numba
import math

@numba.jit(nopython=True,
           parallel=False,
           fastmath=True)
def my_norm_numba(a):
    result = 0
    for i in range(len(a)):
        result += a[i] ** 2
    return math.sqrt(result)

In [ ]:
import numpy as np
x = np.random.rand(int(1e7*8))
y1 = my_norm_numba(x)
y2 = np.linalg.norm(x)
print(y1,y2)

In [ ]:
%timeit _ = my_norm_numba(x)

In [ ]:
%timeit _ = np.linalg.norm(x)

## Přidáme paralelizaci

In [ ]:
import numba
import math

@numba.jit(nopython=True,
           parallel=True,
           fastmath=True)
def my_norm_numba_parallel(a):
    result = 0
    for i in numba.prange(len(a)):
        result += a[i] ** 2
    return math.sqrt(result)

_ = my_norm_numba_parallel(x)

In [ ]:
%timeit _ = my_norm_numba_parallel(x)

Kolik vláken vlastně Numba použije?

In [ ]:
numba.get_num_threads()

Můžeme to zkusit změnit, ale numba dovolí pouze tokik kolik je logických jader.

In [ ]:
numba.set_num_threads(2)

In [ ]:
%timeit _ = my_norm_numba_parallel(x)

In [ ]:
@numba.jit(nopython=True, parallel=True)
def my_dot_numba2(a, b):
    result = 0
    with numba.parallel_chunksize(1000000):
        for i in numba.prange(a.shape[0]):
            result += a[i] * b[i]
    return result
_ = my_dot_numba2(a, b)

In [ ]:
%timeit c = my_dot_numba2(a, b)